# fastbook 14: Implement ResNet

In [2]:
from fastai.vision.all import *

In [7]:
def get_dls(url, presize, resize):
    path = untar_data(url)
    dls = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(valid_name='val'),
        get_y=parent_label,
        item_tfms=Resize(presize),
        batch_tfms=[
            *aug_transforms(min_scale=0.5, size=resize),
            Normalize.from_stats(*imagenet_stats),
        ],
    ).dataloaders(path, bs=128)
    return dls

dls = get_dls(URLs.IMAGENETTE_160, 160, 128)

## What's SOTA ResNet?

In [12]:
import timm
timm.list_models('resnet50*', pretrained=True)

['resnet50', 'resnet50_gn', 'resnet50d']

In [14]:
learn = cnn_learner(dls, 'resnet50')
learn.model

/root/mambaforge/lib/python3.9/site-packages/fastai/vision/learner.py:275: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


Sequential(
  (0): TimmBody(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act2): ReLU(inplace=True)
          (aa): Identity()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fals

## How should we construct the above model efficiently?

* At 1st level, 'resnet50' is composed of a 'stem', 4* 'layer' and a 'head' part
* At 2nd level, a 'layer' is composed of 3, 4, 6 * 'bottleneck' layers
* At 3rd level, a 'bottleneck' is composed of 3 * 'conv' layers

## STEM


## Layler

### Bottleneck

#### Conv

## HEAD